# Clearing the room with a single switch
*The room Dr. Ryoko needs to cross has a floor made of 3 x 3 tiles. Each tile represents a single qubit.<br/>
Some of the qubits change from the ground state to the excited state and vice versa. <br/> You notice that there is a pattern - the floor can only be in either one of the four patterns as shown in each of the examples below. <br/>
To help Dr. Ryoko cross the room safely, you need to find out which board can be cleared with a single switch operation. <br/>*

# Week2-B: Four-Lights Out
In this problem, we are dealing with multiple binary data at the same time. 
We have to determine if each of the given four Lights Out boards are solvable under the given constraints, so let's devise a quantum circuit to solve them all at the same time.

As an example, let's consider how to find a board that can be cleared with just a single switch operation from the 4 boards given below. The initial state of the 4 boards is given in the following two-dimensional array, where "0" and "1" represent "off" and "on" respectively similar to the previous learning problem:

lightsout4_ex=\[\[Board 0\],\[Board 1\],\[Board 2\],\[Board 3\]\]

## Answer Strategy
If only one board is given, this is a decision problem.
Using the algorithm from the first Lights Out puzzle (2A), you can solve this problem by counting the "1"s in the output.
 
If we are given multiple boards, there will be several approaches.
1. Iterate the same "one board algorithm" for each board.
2. Hold information for multiple boards at the same time and solve the problems in a single run (execute the algorithm once). 
- For the rest of this document, we discuss how to use the latter approach to solve this type of problem.

First, how do we keep data for all the boards at the same time?
1. Naive data structures:　　9 Qubits/board * 4 boards > 32 qubits (Upper limit of ibm_qasm_simulator).
2. Prepare the  superposition state:   $\vert Board 0\rangle + \vert Board 1\rangle + \vert Board 2\rangle + \vert Board 3\rangle$.
    - The circuit configuration used for state generation is non-trivial.
3. *qRAM* is known as one solution. 
    - **Pros**: Intuitive implementation. 
    - **Cons**: Computationally expensive. 

Of course you can devise and adopt other smart ways to do this.

Here, we will focus on *qRAM* and describe its configuration and implementation.

In [2]:
from qiskit import *
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import IBMQ, Aer, execute
provider = IBMQ.load_account()

/Users/iuliazidaru/Documents/workspace/quantum-playground/venv/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


## qRAM: Quantum Random Access Memory

In classical computers, RAM (Random Access Memory) is a type of volatile memory that has memory addresses $j$ and stores binary data corresponding to each address $D_j$.

In the case of [qRAM](https://arxiv.org/abs/0708.1879) in a quantum computer, address qubits $a$ have the $N$-addresses as superposition and the corresponding binary data is stored in data qubits $d$ as a state vector.
\\[
\sum_{j}\frac{1}{\sqrt{N}}\vert j \rangle_{a}\vert 0 \rangle_{d}\xrightarrow{qRAM}\sum_{j}\frac{1}{\sqrt{N}}\vert j \rangle_{a}\vert D_{j} \rangle_{d}
\\]　　
We call the right-hand side state "qRAM" and the corresponding gate operation "qRAM operation".

Although qRAM operation requires $\mathcal{O}(N\log N)$ gates, it can be used to create superposition states of binary data intuitively.  

qRAM has previously been applied to various quantum machine learning algorithms such as the HHL algorithm. For this problem, let's apply qRAM to Grover's algorithm.

## Example: Find the data from qRAM
Prepare a qRAM of $n$-addresses in which the numbers $k_0, k_1, .. , k_{n-1}$ are stored in this order.  
Find the address in which the number $m$ is stored using Grover's algorithm.  
- $n = 4$
- $k = [1,2,5,7]$
- $m = 7$

### qRAM operation.
Here we show a circuit example of qRAM.

### qRAM Data Search
To perform Grover's algorithm, we invert the sign of the **address qubit** containing $m$. We also need to initialize the **data qubit** by another qRAM operation before the Diffusion operation,

\begin{align*}
\vert j \rangle_{a}\vert D_{j} \rangle_{d} \vert - \rangle_{f}
\xrightarrow{oracle}  
\left \{
 \begin{array}{l}
-\vert j \rangle_{a}\vert D_{j} \rangle_{d} \vert - \rangle_{f},  D_{j} = m\\
\vert j \rangle_{a}\vert D_{j} \rangle_{d} \vert - \rangle_{f},  D_{j}  \neq m
 \end{array}
 \right.
 \xrightarrow{qRAM}
\left \{
 \begin{array}{l}
-\vert j \rangle_{a}\vert 0 \rangle_{d}\vert - \rangle_{f},  D_{j} = m \\
\vert j \rangle_{a}\vert 0 \rangle_{d}\vert - \rangle_{f},　D_{j}\neq m
 \end{array}
 \right.
 \end{align*}
 
where $f$ denotes the flag qubit.  

In this case, we can configure an oracle operation using the [C3X gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.C3XGate.html#qiskit.circuit.library.C3XGate) . 

Here, we show the whole circuit for our [qRAM example](#qRAM-Example:-Find-the-data-from-qRAM).

### Considerations for qRAM implementation
In the above description we have introduced a naive *qRAM operation* circuit.
Depending on the data structure, we can simplify the circuit by using **gate synthesis** (equivalence transformation) techniques.
Also, some simplified gates, e.g. [RCCX](https://qiskit.org/documentation/stubs/qiskit.circuit.library.RCCXGate.html#qiskit.circuit.library.RCCXGate), may help improve your *CNOT*-saving implementation.

An example of gate synthesis is shown below.

## Learning Exercise II-B
Let's solve a 4-Lights Out problem with qRAM.  

When the initial board state lightsout4=\[\[Board 0\],\[Board 1\],\[Board 2\],\[Board 3\]\] is described by the following data, 
determine the _binary_ number of the solvable boards in $3$ switch operations.  (ex. Board 0 → 00, 1 → 01, 2 → 10, 3 → 11)

Answer by creating a quantum circuit to solve the puzzle shown in the figure below. In the quantum circuit to be submitted, measure only the `solution` (2bit) that solves the puzzle.

To submit your solution, create a function which takes "lightsout4" as an input and return `QuantumCircuit`.  Function's name does not matter. Make sure it works even if you input another dataset to "lightsout4".

 **In addition, please implement the quantum circuit within 28 qubits.**

Please note that you can get the answer with the same endian as the one used in the description. You can also use the following function.
```python
qc = qc.reverse_bits()
```

In [6]:
lightsout4=[[1, 1, 1, 0, 0, 0, 1, 0, 0],[1, 0, 1, 0, 0, 0, 1, 1, 0],[1, 0, 1, 1, 1, 1, 0, 0, 1],[1, 0, 0, 0, 0, 0, 1, 0, 0]]

### Hints
- Change the oracle of [qRAM data search](#qRAM-Data-search) to an appropriate one.
- Data storing/writing in *QRAM operation* can be performed in any order. We can reduce the number of gates by taking into account the _hamming distance_ of the address and input data.

In [7]:
def execute_circuit(circuit):
    backend = Aer.get_backend('qasm_simulator')
    job = execute(circuit, backend=backend, shots=8000, seed_simulator=12345, backend_options={"fusion_enable":True})
    #job = execute(qc, backend=backend, shots=8192)
    result = job.result()
    count = result.get_counts()
    print(count)


In [8]:
def qram(circuit, address_register, data_register):
    # address 0 -> data = 1 (001)
    circuit.x([address_register[0],address_register[1]])
    circuit.ccx(address_register[0],address_register[1],data_register[2])
    circuit.x([address_register[0],address_register[1]])
    circuit.barrier()
    # address 1 -> data = 2 (010)
    circuit.x(address_register[0])
    circuit.ccx(address_register[0],address_register[1],data_register[1])
    circuit.x(address_register[0])
    circuit.barrier()
    # address 2 -> data = 5 (101)
    circuit.x(address_register[1])
    circuit.ccx(address_register[0],address_register[1],data_register[2])
    circuit.ccx(address_register[0],address_register[1],data_register[0])
    circuit.x(address_register[1])
    circuit.barrier()
    # address 3 -> data = 7 (111)
    circuit.ccx(address_register[0],address_register[1],data_register[2])
    circuit.ccx(address_register[0],address_register[1],data_register[1])
    circuit.ccx(address_register[0],address_register[1],data_register[0])
    circuit.barrier()

In [9]:
def qram_reversed(circuit, address_register, data_register):
    # address 3 -> data = 7 (111)
    circuit.ccx(address_register[0],address_register[1],data_register[2])
    circuit.ccx(address_register[0],address_register[1],data_register[1])
    circuit.ccx(address_register[0],address_register[1],data_register[0])
    circuit.barrier()
    # address 2 -> data = 5 (101)
    circuit.x(address_register[1])
    circuit.ccx(address_register[0],address_register[1],data_register[2])
    circuit.ccx(address_register[0],address_register[1],data_register[0])
    circuit.x(address_register[1])
    circuit.barrier()
    # address 1 -> data = 2 (010)
    circuit.x(address_register[0])
    circuit.ccx(address_register[0],address_register[1],data_register[1])
    circuit.x(address_register[0])
    circuit.barrier()
    # address 0 -> data = 1 (001)
    circuit.x([address_register[0],address_register[1]])
    circuit.ccx(address_register[0],address_register[1],data_register[2])
    circuit.x([address_register[0],address_register[1]])
    circuit.barrier()




In [10]:

def in_range(index, max_len):
    if 0 <= index < max_len:
        return True
    return False

from math import sqrt

def flip_tile(qc,flip,tile):
    for i in range(0, len(flip)):
        #print('flip ' + str(i) + 'tile ' + str(i))
        qc.cx(flip[i], tile[i])
        # nit the first cell in row
        if in_range(i-1, len(flip)) and i % sqrt(len(flip)):
            #print('flip ' + str(i) + 'tile ' + str(i-1))
            qc.cx(flip[i], tile[i-1])
        # not the last cell in row
        if in_range(i+1, len(flip)) and (i + 1) % sqrt(len(flip)):
            #print('flip ' + str(i) + 'tile ' + str(i+1))
            qc.cx(flip[i], tile[i+1])
        if in_range(i-sqrt(len(flip)), len(flip)):
            #print('flip ' + str(i) + 'tile ' + str(int(i-sqrt(len(flip)))))
            qc.cx(flip[i], tile[int(i-sqrt(len(flip)))])
        if in_range(i+sqrt(len(flip)), len(flip)):
            #print('flip ' + str(i) + 'tile ' + str(int(i+sqrt(len(flip)))))
            qc.cx(flip[i], tile[int(i+sqrt(len(flip)))])
    #print('\n\nend flip')



In [11]:
def initialize(circuit, address_register, flip_register, oracle):
    circuit.h([address_register[0],address_register[1]])
    circuit.h(flip_register[:3])
    circuit.x(oracle[0])
    circuit.h(oracle[0])


In [12]:
#qram
def qram_exercise(prob_array, qc, qr, qa):
    k=0
    for board in prob_array:
        if k//2==0:
            qc.x(qa[0])
        if k%2==0:
            qc.x(qa[1])
        j=0
        for i in board:
            if i==1:
                qc.ccx(qa[0],qa[1],qr[j])
                j+=1
            else:
                j+=1
        if k//2==0:
            qc.x(qa[0])
        if k%2==0:
            qc.x(qa[1])
        k+=1

In [13]:
#U2A (base: 2a-another solution)
def flip_1(qc,flip,tile):
    # push 0
    qc.cx(flip[0], tile[0])
    qc.cx(flip[0], tile[1])
    qc.cx(flip[0], tile[3])
    # push 1
    qc.cx(flip[1], tile[0])
    qc.cx(flip[1], tile[1])
    qc.cx(flip[1], tile[2])
    qc.cx(flip[1], tile[4])
    # push 2
    qc.cx(flip[2], tile[1])
    qc.cx(flip[2], tile[2])
    qc.cx(flip[2], tile[5])

def inv_1(qc,flip,tile):
    # copy 0,1,2
    qc.cx(tile[0], flip[3])
    qc.cx(tile[1], flip[4])
    qc.cx(tile[2], flip[5])

def flip_2(qc,flip,tile):
    # apply flip[3,4,5]
    qc.cx(flip[3], tile[0])
    qc.cx(flip[3], tile[3])
    qc.cx(flip[3], tile[4])
    qc.cx(flip[3], tile[6])
    qc.cx(flip[4], tile[1])
    qc.cx(flip[4], tile[3])
    qc.cx(flip[4], tile[4])
    qc.cx(flip[4], tile[5])
    qc.cx(flip[4], tile[7])
    qc.cx(flip[5], tile[2])
    qc.cx(flip[5], tile[4])
    qc.cx(flip[5], tile[5])
    qc.cx(flip[5], tile[8])

def inv_2(qc,flip,tile):
    # copy 3,4,5
    qc.cx(tile[3], flip[6])
    qc.cx(tile[4], flip[7])
    qc.cx(tile[5], flip[8])

def flip_3(qc,flip,tile):
    qc.cx(flip[6], tile[3])
    qc.cx(flip[6], tile[6])
    qc.cx(flip[6], tile[7])
    qc.cx(flip[7], tile[4])
    qc.cx(flip[7], tile[6])
    qc.cx(flip[7], tile[7])
    qc.cx(flip[7], tile[8])
    qc.cx(flip[8], tile[5])
    qc.cx(flip[8], tile[7])
    qc.cx(flip[8], tile[8])

def lights_out_oracle(qc,tile,oracle,auxiliary):
    qc.x(tile[6:9])
    qc.mct(tile[6:9], oracle[0], auxiliary)
    qc.x(tile[6:9])

def flipdiffusion(qc,flip):
    qc.h(flip[:3])
    qc.x(flip[:3])
    qc.h(flip[2])
    qc.ccx(flip[0],flip[1],flip[2])
    qc.h(flip[2])
    qc.x(flip[:3])
    qc.h(flip[:3])

In [14]:
# Counter components
def counter(qc, flip,auxiliary):
    for i in range(len(flip)):
        qc.mct([flip[i],auxiliary[0],auxiliary[1],auxiliary[2]],auxiliary[3],auxiliary[4:6])
        qc.mct([flip[i],auxiliary[0],auxiliary[1]],auxiliary[2],auxiliary[4])
        qc.ccx(flip[i],auxiliary[0],auxiliary[1])
        qc.cx(flip[i],auxiliary[0])

def revcounter(qc, flip,auxiliary):
    for i in range(len(flip)):
        qc.cx(flip[i],auxiliary[0])
        qc.ccx(flip[i],auxiliary[0],auxiliary[1])
        qc.mct([flip[i],auxiliary[0],auxiliary[1]],auxiliary[2],auxiliary[4])
        qc.mct([flip[i],auxiliary[0],auxiliary[1],auxiliary[2]],auxiliary[3],auxiliary[4:6])

def addressdiffusion(qc,address):
    qc.h(address[:2])
    qc.x(address[:2])
    qc.cz(address[0],address[1])
    qc.x(address[:2])
    qc.h(address[:2])

In [15]:
def week2b_ans_func(lightout4):
    ##### Build your cirucuit here
    ####  In addition, please make it a function that can solve the problem even with different inputs (lightout4). We do validation with different inputs.
    address = QuantumRegister(2, 'address')
    # data = QuantumRegister(3, 'data')
    tile = QuantumRegister(9, 'tile')
    flip = QuantumRegister(9, 'flip')
    oracle = QuantumRegister(1, 'oracle')
    auxiliary = QuantumRegister(6, 'auxiliary')
    classic = ClassicalRegister(2, 'classic_output')
    qc = QuantumCircuit(address, tile, flip, oracle, auxiliary, classic)

    # address preparation
    initialize(qc, address, flip, oracle)
    qram_exercise(lightsout4, qc, tile, address)
    #U_2A
    for i in range(2):
        flip_1(qc,flip,tile)
        inv_1(qc,flip,tile)
        flip_2(qc,flip,tile)
        inv_2(qc,flip,tile)
        flip_3(qc,flip,tile)

        lights_out_oracle(qc,tile,oracle,auxiliary)

        flip_3(qc,flip,tile)
        inv_2(qc,flip,tile)
        flip_2(qc,flip,tile)
        inv_1(qc,flip,tile)
        flip_1(qc,flip,tile)

        flipdiffusion(qc,flip)


    #Converting "3 qubits flip-information" to 9 qubits.
    flip_1(qc,flip,tile)
    inv_1(qc,flip,tile)
    flip_2(qc,flip,tile)
    inv_2(qc,flip,tile)

    #count number of "1"s
    counter(qc, flip,auxiliary)

    #Inverting the phase of a legal solution (up to 3 pushes).
    qc.x(auxiliary[2])
    qc.x(auxiliary[3])
    qc.ccx(auxiliary[2], auxiliary[3],oracle[0])
    qc.x(auxiliary[2])
    qc.x(auxiliary[3])

    #Uncomputing: The phase information returns to the 3 qubits flip-information.
    revcounter(qc, flip, auxiliary)
    inv_2(qc,flip,tile)
    flip_2(qc,flip,tile)
    inv_1(qc,flip,tile)
    flip_1(qc,flip,tile)

    #Uncomputing: The phase information returns to the 9 qubits board-information.
    for i in range(2):
        flipdiffusion(qc,flip)
        flip_1(qc,flip,tile)
        inv_1(qc,flip,tile)
        flip_2(qc,flip,tile)
        inv_2(qc,flip,tile)
        flip_3(qc,flip,tile)

        lights_out_oracle(qc,tile,oracle,auxiliary)

        flip_3(qc,flip,tile)
        inv_2(qc,flip,tile)
        flip_2(qc,flip,tile)
        inv_1(qc,flip,tile)
        flip_1(qc,flip,tile)

    #Uncomputing: The phase information returns to the QRAM address qubits.
    qram_exercise(lightsout4, qc, tile, address)

    #Diffusion
    addressdiffusion(qc,address)
    # Measurement
    qc.measure(address,classic[0:2])

    # Reverse the output string.
    qc = qc.reverse_bits()

    return qc

In [16]:
circuit = week2b_ans_func(lightsout4)
execute_circuit(circuit)

{'00': 63, '01': 7765, '10': 103, '11': 69}


<ipython-input-7-f580fa7aaf46>:3: DeprecationWarning: Using `backend_options` kwarg has been deprecated as of qiskit-aer 0.7.0 and will be removed no earlier than 3 months from that release date. Runtime backend options should now be added directly using kwargs for each option.
  job = execute(circuit, backend=backend, shots=8000, seed_simulator=12345, backend_options={"fusion_enable":True})


In [17]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

# Unroll the circuit
pass_ = Unroller(['u3', 'cx'])
pm = PassManager(pass_)
new_circuit = pm.run(circuit)

# obtain gates
gates=new_circuit.count_ops()
print(gates)

OrderedDict([('u3', 1964), ('cx', 1951), ('measure', 2)])
